<a href="https://colab.research.google.com/github/zeon62/TinyML/blob/main/structure2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TENSORFLOW VERSION**

In [ ]:
!pip install tensorflow==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
import os

In [ ]:
SAMPLES = 1000
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)
np.random.shuffle(x_values)
y_values = np.sin(x_values)

In [ ]:
y_values += 0.1 * np.random.randn(*y_values.shape)

In [ ]:
TRAIN_SPLIT = int(0.6*SAMPLES)
TEST_SPLIT = int(0.2*SAMPLES + TRAIN_SPLIT)

x_train, x_validate, x_test = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_validate, y_test = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
from tensorflow.keras import layers
model_1 = tf.keras.Sequential()
model_1.add(layers.Dense(16, activation = 'relu', input_shape = (1,)))
model_1.add(layers.Dense(1))
model_1.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
model_1.save("model/model.h5")
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                32        
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [ ]:
type(model_1)

keras.engine.sequential.Sequential

In [ ]:
model_1

In [ ]:
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='model/checkpoint/model.{epoch:02d}.ckpt')
]

In [ ]:
model_1.save_weights('model/checkpoint/model.{epoch:02d}.ckpt'.format(epoch=0))

In [ ]:
history_1 = model_1.fit(x_train, y_train, epochs=10, batch_size=1, validation_data=(x_validate, y_validate))

Epoch 1/10
600/600 [==============================] - 6s 4ms/step - loss: 0.4658 - mae: 0.5858 - val_loss: 0.3330 - val_mae: 0.4973
Epoch 2/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2639 - mae: 0.4483 - val_loss: 0.2357 - val_mae: 0.4170
Epoch 3/10
600/600 [==============================] - 2s 3ms/step - loss: 0.2035 - mae: 0.3960 - val_loss: 0.1960 - val_mae: 0.3838
Epoch 4/10
600/600 [==============================] - 2s 3ms/step - loss: 0.1861 - mae: 0.3716 - val_loss: 0.1853 - val_mae: 0.3710
Epoch 5/10
600/600 [==============================] - 2s 3ms/step - loss: 0.1802 - mae: 0.3631 - val_loss: 0.1786 - val_mae: 0.3589
Epoch 6/10
600/600 [==============================] - 2s 3ms/step - loss: 0.1774 - mae: 0.3547 - val_loss: 0.1788 - val_mae: 0.3480
Epoch 7/10
600/600 [==============================] - 2s 3ms/step - loss: 0.1759 - mae: 0.3489 - val_loss: 0.1739 - val_mae: 0.3510
Epoch 8/10
600/600 [==============================] - 2s 3ms/step - loss: 0.

In [ ]:
model_json = model_1.to_json()
with open('model/model_json.json', 'w') as f:
		f.write(model_json)

**CONVERT**

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_1)
tflite_model = converter.convert()

open("model/sine_model.tflite", "wb").write(tflite_model)

1656

In [ ]:
import sys
sys.stdout = open('model/Analyzer.txt', 'w')
print(tf.lite.experimental.Analyzer.analyze(model_content=tflite_model))

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_1)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_generator():
    for value in x_test:
      yield[np.array(value, dtype = np.float32, ndmin = 2)]
converter.representative_dataset = representative_dataset_generator
tflite_model = converter.convert()

open("model/sine_model_quantized.tflite", "wb").write(tflite_model)

/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


2152

In [ ]:
sys.stdout = open('model/Qnalyzer.txt', 'w')
print(tf.lite.experimental.Analyzer.analyze(model_content=tflite_model))

In [ ]:
basic_model_size = os.path.getsize("model/sine_model.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("model/sine_model_quantized.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

In [ ]:
!apt-get -qq install xxd
!xxd -i model/sine_model.tflite > model/sine_model.cc
!xxd -i model/sine_model_quantized.tflite > model/sine_model_quantized.cc

In [ ]:
!xxd -c 16 model/sine_model_quantized.tflite > model/testq.cc
!xxd -c 16 model/sine_model.tflite > model/test.cc

In [ ]:
!zip -r model.zip model

In [ ]:
from google.colab import files
files.download("model.zip")

#sys.stdout.close()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
type(model_1.call)

method

In [ ]:
model_json

'{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 1], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_2_input"}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "batch_input_shape": [null, 1], "dtype": "float32", "units": 16, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_3", "trainable": true, "dtype": "float32", "units": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, 